In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/ML_project/ml_final.csv")

In [ ]:
df

,movie_id,title,overview,genres,keywords,cast,crew,bert_embeddings
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","['Action', 'Adventure', 'Fantasy', 'ScienceFic...","['cultureclash', 'future', 'spacewar', 'spacec...","['SamWorthington', 'ZoeSaldana', 'SigourneyWea...",['JamesCameron'],[-5.57695508e-01 3.88419092e-01 1.16744876e-...
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","['Adventure', 'Fantasy', 'Action']","['ocean', 'drugabuse', 'exoticisland', 'eastin...","['JohnnyDepp', 'OrlandoBloom', 'KeiraKnightley']",['GoreVerbinski'],[-5.23163915e-01 4.34472561e-01 1.76223695e-...
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"['Action', 'Adventure', 'Crime']","['spy', 'basedonnovel', 'secretagent', 'sequel...","['DanielCraig', 'ChristophWaltz', 'LéaSeydoux']",['SamMendes'],[-5.38812280e-01 4.61531073e-01 1.30909756e-...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"['Action', 'Crime', 'Drama', 'Thriller']","['dccomics', 'crimefighter', 'terrorist', 'sec...","['ChristianBale', 'MichaelCaine', 'GaryOldman']",['ChristopherNolan'],[-8.94423068e-01 3.12668115e-01 -2.38383055e-...
4,49529,John Carter,"John Carter is a war-weary, former military ca...","['Action', 'Adventure', 'ScienceFiction']","['basedonnovel', 'mars', 'medallion', 'spacetr...","['TaylorKitsch', 'LynnCollins', 'SamanthaMorton']",['AndrewStanton'],[-0.7652049 0.39548582 -0.32146108 0.326774...
...,...,...,...,...,...,...,...,...
4801,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,"['Action', 'Crime', 'Thriller']","['unitedstates–mexicobarrier', 'legs', 'arms',...","['CarlosGallardo', 'JaimedeHoyos', 'PeterMarqu...",['RobertRodriguez'],[-6.17380917e-01 4.57069695e-01 -1.80182472e-...
4802,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,"['Comedy', 'Romance']",[],"['EdwardBurns', 'KerryBishé', 'MarshaDietlein']",['EdwardBurns'],[-5.30458093e-01 4.28237736e-01 3.11997950e-...
4803,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...","['Comedy', 'Drama', 'Romance', 'TVMovie']","['date', 'loveatfirstsight', 'narration', 'inv...","['EricMabius', 'KristinBooth', 'CrystalLowe']",['ScottSmith'],[-9.37059700e-01 3.35958928e-01 -2.56385714e-...
4804,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...,[],[],"['DanielHenney', 'ElizaCoupe', 'BillPaxton']",['DanielHsia'],[-7.62437284e-01 3.56536061e-01 -3.34922552e-...


In [ ]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertModel

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


# Assuming your text column is named 'text_column'
text_column_name = 'overview'

# Tokenize the text column
tokenized_texts = df[text_column_name].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

# Pad tokenized sequences to the same length
max_len = max(len(tokens) for tokens in tokenized_texts)
padded_sequences = [tokens + [0] * (max_len - len(tokens)) for tokens in tokenized_texts]

# Convert to PyTorch tensors
input_ids = torch.tensor(padded_sequences)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# Create a new DataFrame column with BERT embeddings
embedding_column_name = 'bert_embeddings'

In [ ]:
import ast
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel

In [ ]:
df["bert_embeddings"]

0       [-5.57695508e-01  3.88419092e-01  1.16744876e-...
1       [-5.23163915e-01  4.34472561e-01  1.76223695e-...
2       [-5.38812280e-01  4.61531073e-01  1.30909756e-...
3       [-8.94423068e-01  3.12668115e-01 -2.38383055e-...
4       [-0.7652049   0.39548582 -0.32146108  0.326774...
                              ...                        
4801    [-6.17380917e-01  4.57069695e-01 -1.80182472e-...
4802    [-5.30458093e-01  4.28237736e-01  3.11997950e-...
4803    [-9.37059700e-01  3.35958928e-01 -2.56385714e-...
4804    [-7.62437284e-01  3.56536061e-01 -3.34922552e-...
4805    [-6.51786685e-01  3.36426020e-01 -3.27587813e-...
Name: bert_embeddings, Length: 4806, dtype: object

In [ ]:
import numpy as np

# Assuming df is your DataFrame and 'bert_embeddings' is the column
def parse_embedding_string(embedding_string):
    # Remove square brackets and split the string by spaces
    values = embedding_string.strip('[]').split()

    # Convert each value to a float and create a NumPy array
    embedding_array = np.array([float(value) for value in values])

    return embedding_array

# Apply the parsing function to the 'bert_embeddings' column
df['bert_embeddings'] = df['bert_embeddings'].apply(parse_embedding_string)


In [ ]:
df["bert_embeddings"]

0       [-0.557695508, 0.388419092, 0.116744876, 0.624...
1       [-0.523163915, 0.434472561, 0.176223695, 0.584...
2       [-0.53881228, 0.461531073, 0.130909756, 0.5174...
3       [-0.894423068, 0.312668115, -0.238383055, 0.28...
4       [-0.7652049, 0.39548582, -0.32146108, 0.326774...
                              ...                        
4801    [-0.617380917, 0.457069695, -0.180182472, 0.31...
4802    [-0.530458093, 0.428237736, 0.31199795, 0.6323...
4803    [-0.9370597, 0.335958928, -0.256385714, 0.2279...
4804    [-0.762437284, 0.356536061, -0.334922552, 0.19...
4805    [-0.651786685, 0.33642602, -0.327587813, 0.247...
Name: bert_embeddings, Length: 4806, dtype: object

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Function to perform similarity search based on movie name
def similarity_search(query_movie_name, df, metric='cosine_similarity'):
    # Get the overview for the query movie
    query_overview = df[df['title'] == query_movie_name]['overview'].values[0]

    # Tokenize the query overview
    query_tokens = tokenizer.encode(query_overview, add_special_tokens=True)

    # Pad tokenized query to the same length
    query_padded = query_tokens + [0] * (max_len - len(query_tokens))

    # Convert to PyTorch tensor
    query_input_ids = torch.tensor([query_padded])

    # Use BERT model to get the embedding for the query
    with torch.no_grad():
        query_embedding = model(query_input_ids).last_hidden_state[:, 0, :].numpy()

    # Flatten the query_embedding
    query_embedding_flat = query_embedding.flatten()

    # Calculate similarity with other movies
    if metric == 'cosine_similarity':
        # Ensure that the dimensions are consistent for cosine_similarity
        query_embedding_flat = query_embedding_flat.reshape(1, -1)
        movie_embeddings_flat = np.vstack(df[embedding_column_name].apply(lambda x: x.flatten()))
        similarities = cosine_similarity(query_embedding_flat, movie_embeddings_flat)
    else:
        # Use Euclidean distance as an alternative
        similarities = -((query_embedding_flat - df[embedding_column_name].apply(lambda x: x.flatten())) ** 2).sum(axis=1)

    # Add similarity scores to the DataFrame
    df['similarity'] = similarities.flatten()

    # Sort by similarity
    similar_movies = df.sort_values(by='similarity', ascending=False)

    return similar_movies[['title', 'overview', 'similarity']]






In [ ]:
movies=df
df

,movie_id,title,overview,genres,keywords,cast,crew,bert_embeddings
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","['Action', 'Adventure', 'Fantasy', 'ScienceFic...","['cultureclash', 'future', 'spacewar', 'spacec...","['SamWorthington', 'ZoeSaldana', 'SigourneyWea...",['JamesCameron'],"[-0.557695508, 0.388419092, 0.116744876, 0.624..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","['Adventure', 'Fantasy', 'Action']","['ocean', 'drugabuse', 'exoticisland', 'eastin...","['JohnnyDepp', 'OrlandoBloom', 'KeiraKnightley']",['GoreVerbinski'],"[-0.523163915, 0.434472561, 0.176223695, 0.584..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"['Action', 'Adventure', 'Crime']","['spy', 'basedonnovel', 'secretagent', 'sequel...","['DanielCraig', 'ChristophWaltz', 'LéaSeydoux']",['SamMendes'],"[-0.53881228, 0.461531073, 0.130909756, 0.5174..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"['Action', 'Crime', 'Drama', 'Thriller']","['dccomics', 'crimefighter', 'terrorist', 'sec...","['ChristianBale', 'MichaelCaine', 'GaryOldman']",['ChristopherNolan'],"[-0.894423068, 0.312668115, -0.238383055, 0.28..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","['Action', 'Adventure', 'ScienceFiction']","['basedonnovel', 'mars', 'medallion', 'spacetr...","['TaylorKitsch', 'LynnCollins', 'SamanthaMorton']",['AndrewStanton'],"[-0.7652049, 0.39548582, -0.32146108, 0.326774..."
...,...,...,...,...,...,...,...,...
4801,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,"['Action', 'Crime', 'Thriller']","['unitedstates–mexicobarrier', 'legs', 'arms',...","['CarlosGallardo', 'JaimedeHoyos', 'PeterMarqu...",['RobertRodriguez'],"[-0.617380917, 0.457069695, -0.180182472, 0.31..."
4802,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,"['Comedy', 'Romance']",[],"['EdwardBurns', 'KerryBishé', 'MarshaDietlein']",['EdwardBurns'],"[-0.530458093, 0.428237736, 0.31199795, 0.6323..."
4803,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...","['Comedy', 'Drama', 'Romance', 'TVMovie']","['date', 'loveatfirstsight', 'narration', 'inv...","['EricMabius', 'KristinBooth', 'CrystalLowe']",['ScottSmith'],"[-0.9370597, 0.335958928, -0.256385714, 0.2279..."
4804,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...,[],[],"['DanielHenney', 'ElizaCoupe', 'BillPaxton']",['DanielHsia'],"[-0.762437284, 0.356536061, -0.334922552, 0.19..."


In [ ]:
word=movies
word

,movie_id,title,overview,genres,keywords,cast,crew,bert_embeddings
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","['Action', 'Adventure', 'Fantasy', 'ScienceFic...","['cultureclash', 'future', 'spacewar', 'spacec...","['SamWorthington', 'ZoeSaldana', 'SigourneyWea...",['JamesCameron'],"[-0.557695508, 0.388419092, 0.116744876, 0.624..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","['Adventure', 'Fantasy', 'Action']","['ocean', 'drugabuse', 'exoticisland', 'eastin...","['JohnnyDepp', 'OrlandoBloom', 'KeiraKnightley']",['GoreVerbinski'],"[-0.523163915, 0.434472561, 0.176223695, 0.584..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"['Action', 'Adventure', 'Crime']","['spy', 'basedonnovel', 'secretagent', 'sequel...","['DanielCraig', 'ChristophWaltz', 'LéaSeydoux']",['SamMendes'],"[-0.53881228, 0.461531073, 0.130909756, 0.5174..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"['Action', 'Crime', 'Drama', 'Thriller']","['dccomics', 'crimefighter', 'terrorist', 'sec...","['ChristianBale', 'MichaelCaine', 'GaryOldman']",['ChristopherNolan'],"[-0.894423068, 0.312668115, -0.238383055, 0.28..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","['Action', 'Adventure', 'ScienceFiction']","['basedonnovel', 'mars', 'medallion', 'spacetr...","['TaylorKitsch', 'LynnCollins', 'SamanthaMorton']",['AndrewStanton'],"[-0.7652049, 0.39548582, -0.32146108, 0.326774..."
...,...,...,...,...,...,...,...,...
4801,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,"['Action', 'Crime', 'Thriller']","['unitedstates–mexicobarrier', 'legs', 'arms',...","['CarlosGallardo', 'JaimedeHoyos', 'PeterMarqu...",['RobertRodriguez'],"[-0.617380917, 0.457069695, -0.180182472, 0.31..."
4802,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,"['Comedy', 'Romance']",[],"['EdwardBurns', 'KerryBishé', 'MarshaDietlein']",['EdwardBurns'],"[-0.530458093, 0.428237736, 0.31199795, 0.6323..."
4803,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...","['Comedy', 'Drama', 'Romance', 'TVMovie']","['date', 'loveatfirstsight', 'narration', 'inv...","['EricMabius', 'KristinBooth', 'CrystalLowe']",['ScottSmith'],"[-0.9370597, 0.335958928, -0.256385714, 0.2279..."
4804,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...,[],[],"['DanielHenney', 'ElizaCoupe', 'BillPaxton']",['DanielHsia'],"[-0.762437284, 0.356536061, -0.334922552, 0.19..."


In [ ]:
bagword=movies
bagword

,movie_id,title,overview,genres,keywords,cast,crew,bert_embeddings
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","['Action', 'Adventure', 'Fantasy', 'ScienceFic...","['cultureclash', 'future', 'spacewar', 'spacec...","['SamWorthington', 'ZoeSaldana', 'SigourneyWea...",['JamesCameron'],"[-0.557695508, 0.388419092, 0.116744876, 0.624..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","['Adventure', 'Fantasy', 'Action']","['ocean', 'drugabuse', 'exoticisland', 'eastin...","['JohnnyDepp', 'OrlandoBloom', 'KeiraKnightley']",['GoreVerbinski'],"[-0.523163915, 0.434472561, 0.176223695, 0.584..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"['Action', 'Adventure', 'Crime']","['spy', 'basedonnovel', 'secretagent', 'sequel...","['DanielCraig', 'ChristophWaltz', 'LéaSeydoux']",['SamMendes'],"[-0.53881228, 0.461531073, 0.130909756, 0.5174..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"['Action', 'Crime', 'Drama', 'Thriller']","['dccomics', 'crimefighter', 'terrorist', 'sec...","['ChristianBale', 'MichaelCaine', 'GaryOldman']",['ChristopherNolan'],"[-0.894423068, 0.312668115, -0.238383055, 0.28..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","['Action', 'Adventure', 'ScienceFiction']","['basedonnovel', 'mars', 'medallion', 'spacetr...","['TaylorKitsch', 'LynnCollins', 'SamanthaMorton']",['AndrewStanton'],"[-0.7652049, 0.39548582, -0.32146108, 0.326774..."
...,...,...,...,...,...,...,...,...
4801,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,"['Action', 'Crime', 'Thriller']","['unitedstates–mexicobarrier', 'legs', 'arms',...","['CarlosGallardo', 'JaimedeHoyos', 'PeterMarqu...",['RobertRodriguez'],"[-0.617380917, 0.457069695, -0.180182472, 0.31..."
4802,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,"['Comedy', 'Romance']",[],"['EdwardBurns', 'KerryBishé', 'MarshaDietlein']",['EdwardBurns'],"[-0.530458093, 0.428237736, 0.31199795, 0.6323..."
4803,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...","['Comedy', 'Drama', 'Romance', 'TVMovie']","['date', 'loveatfirstsight', 'narration', 'inv...","['EricMabius', 'KristinBooth', 'CrystalLowe']",['ScottSmith'],"[-0.9370597, 0.335958928, -0.256385714, 0.2279..."
4804,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...,[],[],"['DanielHenney', 'ElizaCoupe', 'BillPaxton']",['DanielHsia'],"[-0.762437284, 0.356536061, -0.334922552, 0.19..."


In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

# Assuming 'word' is your DataFrame containing movie information and Word2Vec is trained

# Train Word2Vec model using movie overviews
tokenized_overview = word['overview'].apply(lambda x: x.split())
word2vec_model = Word2Vec(sentences=tokenized_overview, vector_size=100, window=5, min_count=1, workers=4)

# Function to calculate document (movie) embeddings by averaging word vectors
def calculate_doc_embeddings(row):
    word_vectors = [word2vec_model.wv[word] for word in row if word in word2vec_model.wv.key_to_index]
    if len(word_vectors) > 0:
        return sum(word_vectors) / len(word_vectors)
    else:
        return [0] * word2vec_model.vector_size  # Return zero vector if no words found in the model

# Function to perform movie recommendations based on Word2Vec similarity
def word2vec_similarity_search(query_movie, df, word2vec_model):
    query_overview = df[df['title'] == query_movie]['overview'].values[0].split()
    query_embeddings = calculate_doc_embeddings(query_overview)

    df['wsimilarity'] = df['overview'].apply(lambda x: cosine_similarity([query_embeddings], [calculate_doc_embeddings(x.split())])[0][0])
    similar_movies = df[df['title'] != query_movie].nlargest(5, 'wsimilarity')  # Change '5' to the number of recommendations needed

    return similar_movies[['movie_id', 'title', 'wsimilarity']]

# Example usage for recommending movies similar to 'Iron Man 2'
query_movie = 'The Avengers'
similar_movies_word2vec = word2vec_similarity_search(query_movie, word, word2vec_model)

# Displaying the Word2Vec-based results with 'wsimilarity' column
print(f"Movies similar to '{query_movie}' (Word2Vec-based with 'wsimilarity' column):")
print(similar_movies_word2vec)


Movies similar to 'The Avengers' (Word2Vec-based with 'wsimilarity' column):
      movie_id               title  wsimilarity
3237     11176    The Muppet Movie     0.999975
281       4982   American Gangster     0.999974
2429     26602        Summer Catch     0.999972
3039     10090    Reno 911!: Miami     0.999972
993      10533  The Scarlet Letter     0.999970


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Function to perform movie recommendations based on BoW similarity for overview
def recommend_similar_movies_based_on_overview(query_movie_id, df):
    query_overview = df[df['movie_id'] == query_movie_id]['overview'].values[0]
    vectorizer = CountVectorizer()
    overview_bow_matrix = vectorizer.fit_transform(df['overview'])

    query_bow = vectorizer.transform([query_overview])
    df['similarity_overview'] = cosine_similarity(query_bow, overview_bow_matrix).flatten()

    similar_movies_overview = df.nlargest(5, 'similarity_overview')  # Change '5' to the number of recommendations needed

    return similar_movies_overview[['movie_id', 'title', 'similarity_overview']]

# Assuming 'bagword' is your DataFrame containing movie information

# Example usage to recommend similar movies based on a query movie title
query_movie_title = 'The Avengers'
query_movie_id = bagword[bagword['title'] == query_movie_title]['movie_id'].values[0]

similar_movies_based_on_overview = recommend_similar_movies_based_on_overview(query_movie_id, bagword)

# Displaying the BoW-based results for overview
print(f"Movies similar to '{query_movie_title}' based on overview (BoW-based):")
print(similar_movies_based_on_overview)


Movies similar to 'The Avengers' based on overview (BoW-based):
      movie_id                                     title  similarity_overview
16       24428                              The Avengers             1.000000
3436     14631  The Work and the Glory II: American Zion             0.611410
3475     57612  Dolphins and Whales: Tribes of the Ocean             0.608226
3894     23618              The Original Kings of Comedy             0.600071
160      82702                How to Train Your Dragon 2             0.598950


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Function to perform movie recommendations based on TF-IDF similarity for overview
def recommend_similar_movies_based_on_overview(query_movie_title, df):
    query_overview = df[df['title'] == query_movie_title]['overview'].values[0]
    vectorizer = TfidfVectorizer()
    overview_tfidf_matrix = vectorizer.fit_transform(df['overview'])

    query_tfidf = vectorizer.transform([query_overview])
    df['similarity_overview'] = cosine_similarity(query_tfidf, overview_tfidf_matrix).flatten()

    similar_movies_overview = df.nlargest(5, 'similarity_overview')  # Change '5' to the number of recommendations needed

    return similar_movies_overview[['movie_id', 'title', 'similarity_overview']]

# Assuming 'word3' is your DataFrame containing movie information

# Example usage to recommend similar movies based on a query movie title
query_movie_title = 'The Avengers'  # Replace with your desired movie title

similar_movies_based_on_overview = recommend_similar_movies_based_on_overview(query_movie_title, df)

# Displaying the TF-IDF-based results for overview
print(f"Movies similar to '{query_movie_title}' based on overview (TF-IDF-based):")
print(similar_movies_based_on_overview)


Movies similar to 'The Avengers' based on overview (TF-IDF-based):
      movie_id                    title  similarity_overview
16       24428             The Avengers             1.000000
7        99861  Avengers: Age of Ultron             0.193911
3315      9388    Thank You for Smoking             0.147555
4130     67373       This Thing of Ours             0.135052
3148    208869                  Plastic             0.129260


In [ ]:
movies.head(475)

In [ ]:
movies['tags'] = movies['genres'].astype(str) + ' ' + movies['keywords'].astype(str) + ' ' + movies['cast'].astype(str) + ' ' + movies['crew'].astype(str)

movies['tags']

new = movies.drop(columns=['genres','keywords','cast','crew'])

new.head()

new['tags']

# Remove quotes and merge lists into a single description
def merge_tags(row):
    # Remove quotes and concatenate the lists into a single string with spaces
    merged_tags = ' '.join(word.strip("'") for word in ' '.join(row.split("] [")).split(", "))
    return merged_tags

# Apply the function to create a new column 'merged_description'
new['tags'] = new['tags'].apply(merge_tags)

new.head()

# Remove brackets and quotes, merge lists into a single line without quotes between words
def clean_tags(row):
    # Remove brackets and quotes, then merge lists into a single line without quotes between words
    merged_tags = ' '.join(word.strip("'") for word in ' '.join(row.split("'] ['")).split("' '"))
    return merged_tags

# Apply the function to create a new column 'cleaned_description'
new['tags'] = new['tags'].apply(clean_tags)

from sklearn.feature_extraction.text import TfidfVectorizer

# Create TfidfVectorizer without specifying stop words
tfidf = TfidfVectorizer()

# Fit and transform your data
vector = tfidf.fit_transform(new['tags']).toarray()

vector.shape

from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)
similarity




array([[1.        , 0.0223913 , 0.01651814, ..., 0.00780294, 0.        ,
        0.        ],
       [0.0223913 , 1.        , 0.01816953, ..., 0.        , 0.        ,
        0.        ],
       [0.01651814, 0.01816953, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.00780294, 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [ ]:
new.head()

,movie_id,title,overview,bert_embeddings,wsimilarity,similarity_overview,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[-0.557695508, 0.388419092, 0.116744876, 0.624...",0.999484,0.025023,['Action Adventure Fantasy ScienceFiction cult...
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[-0.523163915, 0.434472561, 0.176223695, 0.584...",0.996199,0.053119,['Adventure Fantasy Action ocean drugabuse exo...
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[-0.53881228, 0.461531073, 0.130909756, 0.5174...",0.999437,0.041631,['Action Adventure Crime spy basedonnovel secr...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[-0.894423068, 0.312668115, -0.238383055, 0.28...",0.999565,0.050816,['Action Crime Drama Thriller dccomics crimefi...
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[-0.7652049, 0.39548582, -0.32146108, 0.326774...",0.999737,0.087245,['Action Adventure ScienceFiction basedonnovel...


In [ ]:
new[['movie_id', 'title','overview', 'tags']]

,movie_id,title,overview,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",['Action Adventure Fantasy ScienceFiction cult...
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",['Adventure Fantasy Action ocean drugabuse exo...
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,['Action Adventure Crime spy basedonnovel secr...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,['Action Crime Drama Thriller dccomics crimefi...
4,49529,John Carter,"John Carter is a war-weary, former military ca...",['Action Adventure ScienceFiction basedonnovel...
...,...,...,...,...
4801,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,['Action Crime Thriller unitedstates–mexicobar...
4802,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,['Comedy Romance' 'EdwardBurns KerryBishé Mar...
4803,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",['Comedy Drama Romance TVMovie date loveatfirs...
4804,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...,[ 'DanielHenney ElizaCoupe BillPaxton DanielH...


In [ ]:
word2=new

In [ ]:
word3=new

In [ ]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

# Function to calculate document (tags) embeddings by averaging word vectors
def calculate_tag_embeddings(row, word2vec_model_tags):
    word_vectors = [word2vec_model_tags.wv[word] for word in row if word in word2vec_model_tags.wv.key_to_index]
    if len(word_vectors) > 0:
        return sum(word_vectors) / len(word_vectors)
    else:
        return [0] * word2vec_model_tags.vector_size  # Return zero vector if no words found in the model

# Function to perform movie recommendations based on Word2Vec similarity for tags
def recommend_similar_movies_based_on_tags(query_movie_id, df, word2vec_model_tags):
    query_tags = df[df['movie_id'] == query_movie_id]['tags'].values[0]
    query_tags_list = query_tags.split()
    query_embeddings = calculate_tag_embeddings(query_tags_list, word2vec_model_tags)

    df['similarity_tags'] = df['tags'].apply(lambda x: cosine_similarity([query_embeddings], [calculate_tag_embeddings(x.split(), word2vec_model_tags)])[0][0])
    similar_movies_tags = df.nlargest(5, 'similarity_tags')  # Change '5' to the number of recommendations needed

    return similar_movies_tags[['movie_id', 'title', 'similarity_tags']]

# Assuming 'word2' is your DataFrame containing movie information

# Train Word2Vec model using the tokenized 'tags'
tokenized_tags = word2['tags'].apply(lambda x: x.split())
word2vec_model_tags = Word2Vec(sentences=tokenized_tags, vector_size=100, window=5, min_count=1, workers=4)

# Example usage to recommend similar movies based on a query movie title
query_movie_title = 'Incendies'
query_movie_id = word2[word2['title'] == query_movie_title]['movie_id'].values[0]

similar_movies_based_on_tags = recommend_similar_movies_based_on_tags(query_movie_id, word2, word2vec_model_tags)

# Displaying the Word2Vec-based results for tags
print(f"Movies similar to '{query_movie_title}' based on tags (Word2Vec-based):")
print(similar_movies_based_on_tags)


Movies similar to 'Incendies' based on tags (Word2Vec-based):
      movie_id                    title  similarity_tags
3418     46738                Incendies         1.000000
571      16869     Inglourious Basterds         0.991452
3723     36419               After.Life         0.991411
180       2503     The Bourne Ultimatum         0.991089
2728     25208  The Caveman's Valentine         0.991056


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Function to perform movie recommendations based on BoW similarity for tags
def recommend_similar_movies_based_on_tags(query_movie_title, df):
    query_tags = df[df['title'] == query_movie_title]['tags'].values[0]
    vectorizer = CountVectorizer()
    tags_bow_matrix = vectorizer.fit_transform(df['tags'])

    query_bow = vectorizer.transform([query_tags])
    df['similarity_tags'] = cosine_similarity(query_bow, tags_bow_matrix).flatten()

    similar_movies_tags = df.nlargest(5, 'similarity_tags')  # Change '5' to the number of recommendations needed

    return similar_movies_tags[['movie_id', 'title', 'similarity_tags']]

# Assuming 'word3' is your DataFrame containing movie information

# Example usage to recommend similar movies based on a query movie title
query_movie_title = 'Incendies'  # Replace with your desired movie title

similar_movies_based_on_tags = recommend_similar_movies_based_on_tags(query_movie_title, word3)

# Displaying the BoW-based results for tags
print(f"Movies similar to '{query_movie_title}' based on tags (BoW-based):")
print(similar_movies_based_on_tags)


Movies similar to 'Incendies' based on tags (BoW-based):
      movie_id          title  similarity_tags
3418     46738      Incendies         1.000000
4592     43743         Fabled         0.208514
2905     50601  5 Days of War         0.197814
845      12117       Instinct         0.173494
3389     25388   March or Die         0.173494


In [ ]:
def recommend_tfidf(movie):
    index = new[new['title'] == movie].index[0]  # Get the index of the query movie
    distances = sorted(enumerate(similarity[index]), reverse=True, key=lambda x: x[1])

    # Create lists to store movie indices, titles, and similarity scores
    similar_movies_tfidf = []
    similar_movies_titles = []
    similarity_scores_tfidf = []

    for i in distances[1:]:
        movie_index = i[0]  # Get the movie index
        similarity_score = i[1]  # Similarity score for the movie

        # Append movie index and similarity score to respective lists
        similar_movies_tfidf.append(movie_index)
        similarity_scores_tfidf.append(similarity_score)

        # Retrieve movie title using index and append to the list
        movie_title = new.iloc[movie_index]['title']
        similar_movies_titles.append(movie_title)

    # Create a DataFrame with 'index', 'title', and 'similarity' columns
    similar_movies_tfidf_df = pd.DataFrame({
        'index': similar_movies_tfidf,
        'title': similar_movies_titles,
        'similarity': similarity_scores_tfidf
    })

    return similar_movies_tfidf_df

# Example usage
query_movie = 'The Avengers'
similar_movies_bert = similarity_search(query_movie, df)
similar_movies_tfidf_df = recommend_tfidf(query_movie)

# Display the BERT-based results
print(f"Movies similar to '{query_movie}' (BERT-based):")
print(similar_movies_bert[['title', 'similarity']])

# Display TF-IDF-based results with movie indices, titles, and similarity scores
print(f"\nMovies similar to '{query_movie}' (TF-IDF-based) using movie indices and titles:")
print(similar_movies_tfidf_df)


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Movies similar to 'The Avengers' (BERT-based):
                                                  title  similarity
16                                         The Avengers    1.000000
1558                               The First Wives Club    0.992059
4270                                      The Blue Room    0.990010
2926                                       Major League    0.989858
953                           Gremlins 2: The New Batch    0.989193
...                                                 ...         ...
974                                            The Host    0.437865
975                                            The Host    0.437865
3230                                    Namastey London    0.435168
4159                                   Slacker Uprising    0.421925
4574  Rise of the Entrepreneur: The Search for a Bet...    0.352656

[4806 rows x 2 columns]

Movies similar to 'The Avengers' (TF-IDF-based) using movie indices and titles:
      index                    

In [ ]:
def recommend_movies_based_on_combined_similarity(query_movie, similar_movies_bert):
    index = new[new['title'] == query_movie].index[0]  # Get the index of the query movie
    distances = sorted(enumerate(similarity[index]), reverse=True, key=lambda x: x[1])

    # Create lists to store movie indices, titles, similarity scores for both methods, and combined similarity
    similar_movies_titles = []
    similarity_scores_bert = []
    similarity_scores_tfidf = []
    combined_similarity = []

    for i in distances[1:]:
        movie_index = i[0]  # Get the movie index
        similarity_score_tfidf = i[1]  # Similarity score for the movie from TF-IDF

        # Calculate the combined similarity using BERT-based similarity
        similarity_score_bert = similar_movies_bert[similar_movies_bert['title'] == new.iloc[movie_index]['title']]['similarity'].values[0]
        #combined_score = (similarity_score_bert + similarity_score_tfidf) / 2  # Calculate combined similarity
        combined_score = (0.8* similarity_score_bert) + (0.2* similarity_score_tfidf)


        # Append similarity scores and combined similarity to respective lists
        similarity_scores_tfidf.append(similarity_score_tfidf)
        similarity_scores_bert.append(similarity_score_bert)
        combined_similarity.append(combined_score)

        # Retrieve movie title using index and append to the list
        movie_title = new.iloc[movie_index]['title']
        similar_movies_titles.append(movie_title)

    # Create a DataFrame with 'title', 'similarity_bert', 'similarity_tfidf', and 'combined_similarity' columns
    similar_movies_combined_df = pd.DataFrame({
        'title': similar_movies_titles,
        'similarity_bert': similarity_scores_bert,
        'similarity_tfidf': similarity_scores_tfidf,
        'combined_similarity': combined_similarity
    })

    # Sort the DataFrame by 'combined_similarity' in descending order
    similar_movies_combined_df = similar_movies_combined_df.sort_values(by='combined_similarity', ascending=False)

    return similar_movies_combined_df

# Example usage
query_movie = 'The Avengers'
similar_movies_bert = similarity_search(query_movie, df)
recommended_movies = recommend_movies_based_on_combined_similarity(query_movie, similar_movies_bert)

# Display recommended movies based on final combined similarity
print(f"Recommended movies similar to '{query_movie}' based on final combined similarity:")
print(recommended_movies)


Recommended movies similar to 'The Avengers' based on final combined similarity:
                                                  title  similarity_bert  \
0                               Avengers: Age of Ultron         0.969731   
1                            Captain America: Civil War         0.969084   
3                                            Iron Man 2         0.979322   
4                                               Ant-Man         0.981135   
5                    Captain America: The First Avenger         0.978079   
...                                                 ...              ...   
1472                                 Digimon: The Movie         0.482047   
2728                                    Rugrats Go Wild         0.477267   
3605                                    Namastey London         0.435168   
4285                                   Slacker Uprising         0.421925   
4613  Rise of the Entrepreneur: The Search for a Bet...         0.352656   

      